# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']


R. Burn  ->  R. Burn  |  ['R. Burn']
M. Kürster  ->  M. Kürster  |  ['M. Kürster']
K. Lee  ->  K. Lee  |  ['K. Lee']
Arxiv has 74 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.04499


extracting tarball to tmp_2310.04499...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2310.04499/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'headstuff.tex' from 'tmp_2310.04499/headstuff.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure figures/probCol_detSy_sim_0445_0778_0897_0153
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure figures/probCol_detSy_sim_0445_0778_0897_0153 as tmp_2310.05599/./figures/probCol_detSy_sim_0445_0778_0897_0153.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 152 bibliographic references in tmp_2310.05599/Gorrini_et_al_2023.bbl.
syntax error in line 4: '=' expected
Retrieving document from  https://arxiv.org/e-print/2310.05740


extracting tarball to tmp_2310.05740...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2310.05740/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'RN35_tab_phot.tex' from 'tmp_2310.05740/RN35_tab_phot.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.05599-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.05599) | **Planetary companions orbiting the M dwarfs GJ 724 and GJ 3988. A  CARMENES and IRD collaboration**  |
|| P. Gorrini, et al. -- incl., <mark>R. Burn</mark>, <mark>M. Kürster</mark> |
|*Appeared on*| *2023-10-10*|
|*Comments*| *A&A in press*|
|**Abstract**| We report the discovery of two exoplanets around the M dwarfs GJ 724 and GJ 3988 using the radial velocity (RV) method. We obtained a total of 153 3.5 m Calar Alto/CARMENES spectra for both targets and measured their RVs and activity indicators. We also added archival ESO/HARPS data for GJ 724 and infrared RV measurements from Subaru/IRD for GJ 3988. We searched for periodic and stable signals to subsequently construct Keplerian models, considering different numbers of planets, and we selected the best models based on their Bayesian evidence. Gaussian process (GP) regression was included in some models to account for activity signals. For both systems, the best model corresponds to one single planet. The minimum masses are $10.75^{+0.96}_{-0.87}$ and $3.69^{+0.42}_{-0.41}$ Earth-masses for GJ 724 b and GJ 3988 b, respectively. Both planets have short periods (P < 10 d) and, therefore, they orbit their star closely (a < 0.05 au). GJ 724 b has an eccentric orbit (e = $0.577^{+0.055}_{-0.052}$), whereas the orbit of GJ 3988 b is circular. The high eccentricity of GJ 724 b makes it the most eccentric single exoplanet (to this date) around an M dwarf. Thus, we suggest a further analysis to understand its configuration in the context of planetary formation and architecture. In contrast, GJ 3988 b is an example of a common type of planet around mid-M dwarfs. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.05740-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.05740) | **Multicolor Photometry of Tiny Near-Earth Asteroid 2015 RN$_{35}$ Across  a Wide Range of Phase Angles: Possible Mission Accessible A-type Asteroid**  |
|| J. Beniyama, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2023-10-10*|
|*Comments*| *Accepted for publication in the Astronomical Journal. Any comments are welcome*|
|**Abstract**| Studying small near-Earth asteroids is important to understand their dynamical histories and origins as well as to mitigate the damage of the asteroid impact to the Earth. We report the results of multicolor photometry of the tiny near-Earth asteroid 2015 RN$_{35}$ using the 3.8 m Seimei telescope in Japan and the TRAPPIST-South telescope in Chile over 17 nights in 2022 December and 2023 January. We observed 2015 RN$_{35}$ across a wide range of phase angles from 2$^{\circ}$ to 30$^{\circ}$ in the $g$, $r$, $i$, and $z$ bands in the Pan-STARRS system. These lightcurves show that 2015 RN$_{35}$ is in a non-principal axis spin state with two characteristic periods of $1149.7\pm0.3$ s and $896.01\pm0.01$ s. We found that a slope of a visible spectrum of 2015 RN$_{35}$ is as red as asteroid (269) Justitia, one of the very red objects in the main belt, which indicates that 2015 RN$_{35}$ can be classified as an A- or Z-type asteroid. In conjunction with the shallow slope of the phase curve, we suppose that 2015 RN$_{35}$ is a high-albedo A-type asteroid. We demonstrated that surface properties of tiny asteroids could be well constrained by intensive observations across a wide range of phase angles. 2015 RN$_{35}$ is a possible mission accessible A-type near-Earth asteroid with a small $\Delta v$ of 11.801 km s$^{-1}$ in the launch window between 2030 and 2035. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.04499-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.04499) | **Exploring chemical enrichment of the intracluster medium with the Line  Emission Mapper**  |
|| F. Mernier, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2023-10-10*|
|*Comments*| *19 pages. White paper for a mission concept to be submitted for the 2023 NASA Astrophysics Probes opportunity*|
|**Abstract**| Synthesized in the cores of stars and supernovae, most metals disperse over cosmic scales and are ultimately deposited well outside the gravitational potential of their host galaxies. Since their presence is well visible through their X-ray emission lines in the hot gas pervading galaxy clusters, measuring metal abundances in the intracluster medium (ICM) offers us a unique view of chemical enrichment of the Universe as a whole. Despite extraordinary progress in the field thanks to four decades of X-ray spectroscopy using CCD (and gratings) instruments, understanding the precise stellar origins of the bulk of metals, and when the latter were mixed on Mpc scales, requires an X-ray mission capable of spatial, non-dispersive high resolution spectroscopy covering at least the soft X-ray band over a large field of view. In this White Paper, we demonstrate how the Line Emission Mapper (LEM) probe mission concept will revolutionize our current picture of the ICM enrichment. Specifically, we show that LEM will be able to (i) spatially map the distribution of ten key chemical elements out to the virial radius of a nearby relaxed cluster and (ii) measure metal abundances in serendipitously discovered high-redshift protoclusters. Altogether, these key observables will allow us to constrain the chemical history of the largest gravitationally bound structures of the Universe. They will also solve key questions such as the universality of the initial mass function (IMF) and the initial metallicity of the stellar populations producing these metals, as well as the relative contribution of asymptotic giant branch (AGB) stars, core-collapse, and Type Ia supernovae to enrich the cosmic web over Mpc scales. Concrete observing strategies are also briefly discussed. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2310.05599.md
    + _build/html/tmp_2310.05599/./figures/GLS_activity_selected_J18409-133_1.png
    + _build/html/tmp_2310.05599/./figures/GLS_activity_selected_J18409-133_2.png
    + _build/html/tmp_2310.05599/./figures/J18409-133_detection_map.png
    + _build/html/tmp_2310.05599/./figures/Phased_RV_P1_5d.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\sherlock}{{\fontfamily{pcr}\selectfont SHERLOCK} }$
$\newcommand{\matrixtk}{{\fontfamily{pcr}\selectfont MATRIX} }$
$\newcommand{\TODO}[1]{\textcolor{magenta}{\textsc{todo:} \textit{#1}}}$
$\newcommand{\autoref}$
$\newcommand{\equationautorefname}{Eq.}$
$\newcommand{\figureautorefname}{Fig.}$
$\newcommand{\sectionautorefname}{Sect.}$
$\newcommand{\subsectionautorefname}{Sect.}$
$\newcommand{\subsubsectionautorefname}{Sect.}$</div>



<div id="title">

# Planetary companions orbiting the M dwarfs GJ 724 and GJ 3988

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.05599-b31b1b.svg)](https://arxiv.org/abs/2310.05599)<mark>Appeared on: 2023-10-10</mark> -  _A&A in press_

</div>
<div id="authors">

P. Gorrini, et al. -- incl., <mark>R. Burn</mark>, <mark>M. Kürster</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of two exoplanets around the M dwarfs GJ 724 and GJ 3988 using the radial velocity (RV) method. We obtained a total of 153 3.5 m Calar Alto/CARMENES spectra for both targets and measured their RVs and activity indicators. We also added archival ESO/HARPS data for GJ 724 and infrared RV measurements from Subaru/IRD for GJ 3988. We searched for periodic and stable signals to subsequently construct Keplerian models, considering different numbers of planets, and we selected the best models based on their Bayesian evidence. Gaussian process (GP) regression was included in some models to account for activity signals. For both systems, the best model corresponds to one single planet. The minimum masses are $10.75^{+0.96}_{-0.87}$ and $3.69^{+0.42}_{-0.41}$ Earth-masses for GJ 724 b and GJ 3988 b, respectively. Both planets have short periods ( $P < \SI{10}{\day}$ ) and, therefore, they orbit their star closely ( $a < \SI{0.05}{\astronomicalunit}$ ).  GJ 724 b has an eccentric orbit ( $e = 0.577^{+0.055}_{-0.052}$ ), whereas the orbit of GJ 3988 b is circular. The high eccentricity of GJ 724 b makes it the most eccentric single exoplanet (to this date) around an M dwarf. Thus, we suggest a further analysis to understand its configuration in the context of planetary formation and architecture. In contrast, GJ 3988 b is an example of a common type of planet around mid-M dwarfs.

</div>

<div id="div_fig1">

<img src="tmp_2310.05599/./figures/GLS_activity_selected_J18409-133_1.png" alt="Fig18.1" width="50%"/><img src="tmp_2310.05599/./figures/GLS_activity_selected_J18409-133_2.png" alt="Fig18.2" width="50%"/>

**Figure 18. -** GLS periodograms of the (combined) photometric data and spectroscopic activity indicators from GJ 724 with peaks that reach at least a FAP level of 10\% in the period range of \SI{2}{\day} to \SI{200}{\day}. The gray dashed horizontal lines correspond to the FAP levels of 0.1\%, 1\%, and 10\%(from top to bottom, respectively). The mean rotation period of \SI{56}{\day} determined from the photometry is marked by the blue dashed line, while its second harmonic (${P_{rot}}/2$) of \SI{28}{\day} is depicted by the orange dashed line. The period of the 5.1-day planet is highlighted by the red solid line. (*fig:activity_GLS_GJ724*)

</div>
<div id="div_fig2">

<img src="tmp_2310.05599/./figures/J18409-133_detection_map.png" alt="Fig7" width="100%"/>

**Figure 7. -** Detection limit map of the RVs of GJ 724. The colour map represents the detection probability of the period-mass combination. The Grid is $50 \times 50$ in mass and period, with 50 phase samples for each combination. The blue star indicates the planet GJ 724 b.  (*fig:GJ724_detection_map*)

</div>
<div id="div_fig3">

<img src="tmp_2310.05599/./figures/Phased_RV_P1_5d.png" alt="Fig3" width="100%"/>

**Figure 3. -** Phased RVs for GJ 724 b from the best fit model ($\text{1P}_\text{(5 d-ecc)} + \text{dSHO-GP}_\text{28 d}$). The red dots show the CARMENES data while the teal squares depicts the HARPS data. The black lines show the median of \num{10000} samples from the posterior. The residuals after subtracting the median model are shown in the lower panel. (*fig:phasefolded_GJ724*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.05599"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

289  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
